<a href="https://colab.research.google.com/github/adidror005/youtube-videos/blob/main/Getting_Bar_Data_for_Many_Tickers_at_Once_Actual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Polygon.IO
* Polygon.IO is a super duper data platform [Polygon.IO](https://polygon.io/)

* Go to Pricing and subscribe to Free Account
    - 4 types of subscriptions for each Options, Stocks, Indices, Forex
    - Free tier is good enough for a lot. 5 API calls max per minute
* For docs go to docs / client library Python. See [Polygon.IO Docs](https://polygon.io/docs/stocks/getting-started)

### Prerequisite Video
### Get Historical Stock Data From Polygon.IO API (click to view)


[![IMAGE ALT TEXT HERE](https://img.youtube.com/vi/6TeHBXbvkPI/0.jpg)](https://www.youtube.com/watch?v=6TeHBXbvkPI)


#### Install Client Library


In [1]:
!pip install polygon-api-client

### API Key
* Get in dashboard https://polygon.io/dashboard
* Copy and paste here to MY_API_KEY. Note I did the "proper" thing by setting a secret, but this isn't necessary and you can just set MY_API_KEY to your API Key string

In [2]:
from google.colab import userdata
MY_API_KEY = userdata.get('POLYGON_API_KEY')

#### Connect to client

In [3]:
from polygon import RESTClient
client = RESTClient(MY_API_KEY)

### Use *get_grouped_daily_aggs* to get OHLCV data for all stocks!
* Non pagenated so don't need to set a limit param

In [4]:
grouped = client.get_grouped_daily_aggs(date='2024-02-14')
len(grouped)

10501

In [5]:
grouped[0:3]

[GroupedDailyAgg(ticker='ASND', open=145.31, high=148.76, low=142.72, close=146.61, volume=441037, vwap=146.1857, timestamp=1707944400000, transactions=7463, otc=None),
 GroupedDailyAgg(ticker='TMFS', open=30.24, high=30.68, low=30.11, close=30.68, volume=11928, vwap=30.2186, timestamp=1707944400000, transactions=82, otc=None),
 GroupedDailyAgg(ticker='BWEN', open=2.52, high=2.59, low=2.44, close=2.56, volume=59086, vwap=2.5259, timestamp=1707944400000, transactions=586, otc=None)]

#### Create Pandas DataFrame

In [6]:
import pandas as pd
df = pd.DataFrame(grouped)
df.head()

,ticker,open,high,low,close,volume,vwap,timestamp,transactions,otc
0,ASND,145.3100,148.760,142.7200,146.6100,441037.0,146.1857,1707944400000,7463.0,None
1,TMFS,30.2400,30.680,30.1100,30.6800,11928.0,30.2186,1707944400000,82.0,None
2,BWEN,2.5200,2.590,2.4400,2.5600,59086.0,2.5259,1707944400000,586.0,None
3,STEM,3.1500,3.345,3.1000,3.3300,2535368.0,3.2623,1707944400000,12410.0,None
4,AULT,0.4928,0.510,0.4168,0.4708,6042618.0,0.4761,1707944400000,15249.0,None


* Since timestamp is ugly let's convert to pandas datatime

In [7]:
df['timestamp']=pd.to_datetime(df['timestamp'],unit='ms')
df.head()

,ticker,open,high,low,close,volume,vwap,timestamp,transactions,otc
0,ASND,145.3100,148.760,142.7200,146.6100,441037.0,146.1857,2024-02-14 21:00:00,7463.0,None
1,TMFS,30.2400,30.680,30.1100,30.6800,11928.0,30.2186,2024-02-14 21:00:00,82.0,None
2,BWEN,2.5200,2.590,2.4400,2.5600,59086.0,2.5259,2024-02-14 21:00:00,586.0,None
3,STEM,3.1500,3.345,3.1000,3.3300,2535368.0,3.2623,2024-02-14 21:00:00,12410.0,None
4,AULT,0.4928,0.510,0.4168,0.4708,6042618.0,0.4761,2024-02-14 21:00:00,15249.0,None


In [9]:
df.shape[0]

10501

#### Can include over the counter traded stocks (OTC) with include_otc=True

In [8]:
grouped = client.get_grouped_daily_aggs(
    "2024-02-16",include_otc=True,
)
len(grouped)

14598

### Get Daily Prices for Every Stock Ticker
##### Suppose we want to get daily prices for many years
* Iterate over dates to create a list of dataframes
* Since we are using the free version, every 5 API calls, we take a 1 minute pause, due to the limit.
* We can use something like pandas_market_calendar to get the days market is open, but this might not be fully accurate for all exchanges.
* Combined the dataframes at the end with pd.concat!


###### *pandas_market_calendar*
Get last 10 trading dates using pandas market calendar. Increase this if you want more of course

In [10]:
!pip install pandas_market_calendars

import pandas_market_calendars as mcal
from datetime import datetime, timedelta

# Define the stock market calendar (NYSE in this case)
nyse = mcal.get_calendar('NYSE')

# Calculate the start and end dates for the last two years
end_date = datetime.now()
start_date = end_date - timedelta(days=10)

# Get the schedule of trading days between the start and end dates
schedule = nyse.schedule(start_date=start_date, end_date=end_date)

# Extract the trading dates
trading_dates = schedule.index.astype(str).tolist()

trading_dates

['2024-06-10',
 '2024-06-11',
 '2024-06-12',
 '2024-06-13',
 '2024-06-14',
 '2024-06-17',
 '2024-06-18']

### Get daily bars for all stock tickers!




In [11]:
import time
df_list = []
for i,date in enumerate(trading_dates):
  if (i % 5 == 0) and (i>0):
    print("Pausing for 1 minute")
    time.sleep(60)
  print(f"Getting data for {date}")
  grouped = client.get_grouped_daily_aggs(date,include_otc=True)
  df = pd.DataFrame(grouped)
  df['timestamp']=pd.to_datetime(df['timestamp'],unit='ms')
  df_list.append(df)

Getting data for 2024-06-10
Getting data for 2024-06-11
Getting data for 2024-06-12
Getting data for 2024-06-13
Getting data for 2024-06-14
Pausing for 1 minute
Getting data for 2024-06-17
Getting data for 2024-06-18


In [12]:
len(df_list)

7

In [14]:
df = pd.concat(df_list)
df.shape

(101581, 10)

In [15]:
df.head()

,ticker,open,high,low,close,volume,vwap,timestamp,transactions,otc
0,REFG,0.000001,0.000001,0.000001,0.000001,2000.0,0.000001,2024-06-10 20:00:00,1.0,True
1,HYEM,19.070000,19.086100,19.040000,19.070000,36666.0,19.064100,2024-06-10 20:00:00,270.0,None
2,UZE,21.230000,21.570000,21.150100,21.500000,88271.0,21.375600,2024-06-10 20:00:00,580.0,None
3,CX,6.630000,6.750000,6.545000,6.680000,6560303.0,6.676300,2024-06-10 20:00:00,23891.0,None
4,PUMSY,5.060000,5.060000,4.970100,5.002000,18404.0,5.004800,2024-06-10 20:00:00,55.0,True


In [16]:
df['timestamp']=pd.to_datetime(df['timestamp'],unit='ms')
df.head()

,ticker,open,high,low,close,volume,vwap,timestamp,transactions,otc
0,REFG,0.000001,0.000001,0.000001,0.000001,2000.0,0.000001,2024-06-10 20:00:00,1.0,True
1,HYEM,19.070000,19.086100,19.040000,19.070000,36666.0,19.064100,2024-06-10 20:00:00,270.0,None
2,UZE,21.230000,21.570000,21.150100,21.500000,88271.0,21.375600,2024-06-10 20:00:00,580.0,None
3,CX,6.630000,6.750000,6.545000,6.680000,6560303.0,6.676300,2024-06-10 20:00:00,23891.0,None
4,PUMSY,5.060000,5.060000,4.970100,5.002000,18404.0,5.004800,2024-06-10 20:00:00,55.0,True


#### Optional:
* Create a Multi-Index
* Save to CSV

In [17]:
df.set_index(['ticker','timestamp'],inplace=True)
df.head()

,,open,high,low,close,volume,vwap,transactions,otc
ticker,timestamp,,,,,,,,
REFG,2024-06-10 20:00:00,0.000001,0.000001,0.000001,0.000001,2000.0,0.000001,1.0,True
HYEM,2024-06-10 20:00:00,19.070000,19.086100,19.040000,19.070000,36666.0,19.064100,270.0,None
UZE,2024-06-10 20:00:00,21.230000,21.570000,21.150100,21.500000,88271.0,21.375600,580.0,None
CX,2024-06-10 20:00:00,6.630000,6.750000,6.545000,6.680000,6560303.0,6.676300,23891.0,None
PUMSY,2024-06-10 20:00:00,5.060000,5.060000,4.970100,5.002000,18404.0,5.004800,55.0,True


In [18]:
df.to_csv('daily_bars_for_all_ticker.csv')